# Data Preprocessing for Machine Learning Models

## 1. Introduction
### Dataset Description
- **Dataset Source**: REMS data collected by the Curiosity Rover. It contains **3,197 records** spanning multiple Martian years, with variables covering **temperature, pressure, UV radiation, and day length**.
- **Key Columns**:
  - `Ls`: Solar longitude representing Mars’ position in its orbit.
  - `sunrise`, `sunset`: Times for sunrise and sunset. 
  - `max_ground_temp`, `min_ground_temp`: Ground temperature extremes.
  - `max_air_temp`, `min_air_temp`: Air temperature extremes.
  - `avg_air_temp`, `avg_ground_temp`: Average air and ground temperatures.
  - `mars_month`: Martian month based on solar longitude (30  degrees per month)
  - `mars_year`: Martian year based on mission start (Initial year = 1)
  - `mars_season`: Martian season based on solar longitude. Curiosity is located in the southern hemisphere which means our seasons are inverted.
  - `day_length`: Length of the Martian day in minutes.
  - `mean_pressue`: Average atmospheric pressure for a given day.
  - `UV_Radiation`: UV index categories.
- Purpose: This dataset helps study Martian weather patterns and seasonal variations.

### Objectives for Preprocessing
 - Encode solar longitude with sin/cos to capture wrap around nature.
 - Create seperate synchronized datasets for scaled and unscaled data for different ML models.
 - Create dummy variables for categorical features: `Season`, `UV Radiation`
 - Use MinMaxScaler() on numerical features: `Mean Air Temp`, `Mean Ground Temp`, `Mean Pressue`, `Day Length`, `Solar Longitude`
 - Create train/test splits for each dataset.

---

## 2. Data Overview

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import joblib
import os

data_path = "../data/"

### 2.1 Load and Inspect Data

In [2]:
# Import data from CSV
df = pd.read_csv(os.path.join(data_path, "cleaned/mars_weather_cleaned.csv"), index_col='sol_number')

# Check that data loaded properly
df.head()

,earth_date_time,Ls,mars_month,mars_year,mars_season,sunrise,sunset,day_length,avg_ground_temp,max_ground_temp,min_ground_temp,avg_air_temp,max_air_temp,min_air_temp,mean_pressure,UV_Radiation
sol_number,,,,,,,,,,,,,,,,
1,2012-08-07,150.0,6,1,winter,05:30:00,17:22:00,712.0,-45.5,-16.0,-75.0,-37.5,8.0,-83.0,739.0,4
9,2012-08-15,155.0,6,1,winter,05:28:00,17:22:00,714.0,-45.5,-16.0,-75.0,-37.5,8.0,-83.0,739.0,4
10,2012-08-16,155.0,6,1,winter,05:28:00,17:22:00,714.0,-45.5,-16.0,-75.0,-37.5,8.0,-83.0,739.0,4
11,2012-08-17,156.0,6,1,winter,05:28:00,17:21:00,713.0,-43.5,-11.0,-76.0,-37.0,9.0,-83.0,740.0,4
12,2012-08-18,156.0,6,1,winter,05:28:00,17:21:00,713.0,-47.0,-18.0,-76.0,-37.0,8.0,-82.0,741.0,4


### **2.2 Summary Statistics**

In [3]:
df.describe()

,Ls,mars_month,mars_year,day_length,avg_ground_temp,max_ground_temp,min_ground_temp,avg_air_temp,max_air_temp,min_air_temp,mean_pressure,UV_Radiation
count,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000
mean,166.959962,6.083203,3.507038,718.169221,-44.097279,-13.182828,-75.011730,-39.147357,2.011730,-80.306537,828.997028,2.583359
std,104.356771,3.466172,1.500218,12.036363,7.401663,10.489177,5.529929,7.262277,9.398862,8.824723,57.224328,0.692132
min,0.000000,1.000000,1.000000,702.000000,-72.500000,-67.000000,-100.000000,-75.500000,-61.000000,-136.000000,702.000000,1.000000
25%,78.000000,3.000000,2.000000,707.000000,-50.500000,-23.000000,-79.000000,-45.500000,-6.000000,-86.000000,785.000000,2.000000
50%,156.000000,6.000000,3.000000,716.000000,-43.000000,-12.000000,-75.000000,-38.000000,3.000000,-80.000000,844.000000,3.000000
75%,254.000000,9.000000,5.000000,730.000000,-37.500000,-4.000000,-71.000000,-33.000000,10.000000,-75.000000,873.000000,3.000000
max,359.000000,12.000000,6.000000,738.000000,-26.500000,11.000000,-52.000000,-1.500000,24.000000,-8.000000,925.000000,4.000000


---

## 3. Data Processing

Many of the features contain fundamentally similar information and will be dropped at this stage.

In [4]:
# Splitting features
cat_feats = ['mars_season', 'UV_Radiation']
num_feats = ['avg_air_temp', 'avg_ground_temp', 'mean_pressure', 'day_length', 'Ls']

# Remapping UV Radiation from 1–4 → string labels
uv_map = {1: "low", 2: "moderate", 3: "high", 4: "very_high"}

df["UV_Radiation"] = (
    df["UV_Radiation"]
    .map(uv_map)          # replace integers with strings
    .astype("category")   # optional: store as pandas categorical
)

# Encoding Cyclical nature in Solar Longitude
df['sin_lon'] = np.sin(2 * np.pi * df['Ls'] / 360)
df['cos_lon'] = np.cos(2 * np.pi * df['Ls'] / 360)
num_feats.remove('Ls')
num_feats.extend(['sin_lon', 'cos_lon'])

# Removing unnecessary features from dataframe
df = df[num_feats + cat_feats]

In [5]:
# Encoding categorical features
df_dummies = pd.get_dummies(df[cat_feats], drop_first=True).astype(int)
df_unscaled = pd.concat([df[num_feats], df_dummies], axis=1)

df_unscaled.head()

,avg_air_temp,avg_ground_temp,mean_pressure,day_length,sin_lon,cos_lon,mars_season_spring,mars_season_summer,mars_season_winter,UV_Radiation_low,UV_Radiation_moderate,UV_Radiation_very_high
sol_number,,,,,,,,,,,,
1,-37.5,-45.5,739.0,712.0,0.500000,-0.866025,0,0,1,0,0,1
9,-37.5,-45.5,739.0,714.0,0.422618,-0.906308,0,0,1,0,0,1
10,-37.5,-45.5,739.0,714.0,0.422618,-0.906308,0,0,1,0,0,1
11,-37.0,-43.5,740.0,713.0,0.406737,-0.913545,0,0,1,0,0,1
12,-37.0,-47.0,741.0,713.0,0.406737,-0.913545,0,0,1,0,0,1


In [6]:
# Scaling numeric features
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(df[num_feats])
df_numeric = pd.DataFrame(scaled_values, columns=num_feats, index=df.index)

df_scaled = pd.concat([df_numeric, df_dummies], axis=1)

df_scaled.head()

joblib.dump(scaler, os.path.join(data_path,"processed/scaler.pkl"))

['../data/processed/scaler.pkl']

In [7]:
# Final shapes
print("Unscaled:", df_unscaled.shape)
print("Scaled:", df_scaled.shape)

# save scaled and unscaled dataframes as CSV
df_unscaled.to_csv(os.path.join(data_path,"processed/processed_weather_unscaled.csv"), index=True) # For later use in SARIMAX model
df_scaled.to_csv(os.path.join(data_path,"processed/processed_weather_scaled.csv"), index=True) # For later use in LSTM and Isolation Forest model

Unscaled: (3197, 12)
Scaled: (3197, 12)


---

## 4. Splitting into Training and Test Sets

In [8]:
# Set split ratio and index
split_ratio = 0.8
split_idx = int(len(df_scaled) * split_ratio)

# --- Scaled Data ---
X_scaled_train = df_scaled.iloc[:split_idx]
X_scaled_test = df_scaled.iloc[split_idx:]

# --- Unscaled Data ---
X_unscaled_train = df_unscaled.iloc[:split_idx]
X_unscaled_test = df_unscaled.iloc[split_idx:]

In [9]:
# Save scaled splits
X_scaled_train.to_csv(os.path.join(data_path, "processed/scaled_train.csv"), index=True)
X_scaled_test.to_csv(os.path.join(data_path, "processed/scaled_test.csv"), index=True)

# Save unscaled splits
X_unscaled_train.to_csv(os.path.join(data_path, "processed/unscaled_train.csv"), index=True)
X_unscaled_test.to_csv(os.path.join(data_path, "processed/unscaled_test.csv"), index=True)

---

## 5. Preprocessing Summary

- Cleaned and structured dataset prepared for time-series forecasting and anomaly detection.
- Categorical features (`mars_season`, `UV_Radiation`) encoded using one-hot encoding.
- Numerical features scaled using `MinMaxScaler` to support LSTM and Isolation Forest models.
- Created two datasets:
  - `df_unscaled`: original numeric values + dummy features (for SARIMAX)
  - `df_scaled`: scaled numeric values + dummy features (for LSTM, Isolation Forest)
- Time-aware 80/20 split applied:
  - `scaled_train.csv`, `scaled_test.csv`
  - `unscaled_train.csv`, `unscaled_test.csv`
- All files saved to: `../data/processed`
- Scaler object saved as `scaler.pkl` for inverse transforms during inference

In [10]:
# Final info summary
print("Preprocessing Complete")
print(f"Scaled train shape   : {X_scaled_train.shape}")
print(f"Scaled test shape    : {X_scaled_test.shape}")
print(f"Unscaled train shape : {X_unscaled_train.shape}")
print(f"Unscaled test shape  : {X_unscaled_test.shape}")
print(f"All files saved to   : {data_path}processed/")

Preprocessing Complete
Scaled train shape   : (2557, 12)
Scaled test shape    : (640, 12)
Unscaled train shape : (2557, 12)
Unscaled test shape  : (640, 12)
All files saved to   : ../data/processed/
